# Splines in Gustaf
## The Library
### Gustaf
> Loyal butler for numerical-analysis-geometry processing 
Gustaf provides many functionalities for meshing, spline-construction and manipulation and visualisation. With regards to splines, this includes:
- Meshing of splines
- Visualization of splines
- Construction through extrusion and revolutions

Gustafs dependencies include `uff` and`splinepy`. It visualizes data using `vedo`.

### Splinepy
> Python N-dimensional Bezier, Rational Bezier, BSpline and NURBS library with C++ backend 
It provides all low-level functionality for splines, including
 - import / export for various formats, such as Irit, XML, iges, JSON and MFEM
 - Order Elevation
 - Evaluation
 - Knot Insertion (NURBS, BSplines)
 - Derivatives (wip)
 - Compositions of Bezier-Extracted elements
 - Multiplication and Addition of Bezier-Extracted elements
 
Splinepy depends on a modified version of `SplineLib`, `Napf` and `Bezman`. Here, `SplineLib` is used for NURBS and BSplines, `Bezman` handles all things Bezier.
 
### Bezman
> Lightweight library in C++ to simplify the composition of Bezier-splines 

Bezman is the `C++` library to provide functionality for the manipulation of Bezier-Splines, including Compositions, Multiplications, and Additions. It also provides functions to facilitate the export of MFEM meshes. Bezman is very templatized, for the purpose of evaluating derivatives and also supports AD-Types, tensors, complex numbers, etc..

## Getting Started
### Basics


In [1]:
import gustaf as gus
import numpy as np

All Spline-Types are constructed using (positional or) the following keyword arguments:
- `degrees` (list)
- `control_points` (list[list] oder `numpy` type)
- `knot_vectors` (list[list])
- `weights`(list of `numpy`-types)

The keywords are used where applicable, i.e.:

| | NURBS | BSpline | Bezier | Rational Bezier |
| -: | :-: | :-: | :-: | :-: |
| `degrees` | X | X | X | X |
| `control_points` | X | X | X | X |
| `knot_vectors` | X | X | - | - |
| `weights` | X | - | - | X |

Let's create some splines:

In [2]:
# Bezier types
bezier_line = gus.Bezier(
    degrees=[1],
    control_points=[[0.,0.],[2,1]]
)
bezier_surface = gus.Bezier(
    degrees=[1,1],
    control_points=[[0.,1.],[2,3],[0,2],[2,4]]
)

# NURBS
nurbs_line= gus.NURBS(
    degrees=[2],
    knot_vectors=[[0,0,0,.5,1,1,1]],
    control_points=[[0.,1.], [2,3],[0,2],[2,4]],
    weights=[1, .5, .8, 1]
)

They can be visualized using the simple show-command

In [3]:
bezier_line.show()

Or Multiple splines at the same time using

In [4]:
# Beside each other
gus.show.show_vedo(bezier_line, bezier_surface, nurbs_line)
# In the same window
gus.show.show_vedo([bezier_line, bezier_surface, nurbs_line])

The transformation into a type with more information is also possible using properties

| FROM \ TO| NURBS | BSpline | Bezier | Rational Bezier |
| -: | :-: | :-: | :-: | :-: |
| NURBS | X | X | X | X |
| BSpline | - | X | X | - |
| Bezier | - | - | X | - |
| Rational Bezier | - | - | X | X |

using

In [5]:
# Change Type
nurbs_from_bezier_line = bezier_line.nurbs
# Visualize
gus.show.show_vedo(nurbs_from_bezier_line, bezier_line)

However, Bezier-patches can be extracted using

In [6]:
# Extract
rational_bezier_list = nurbs_line.extract.beziers()
# Show
gus.show.show_vedo(rational_bezier_list, nurbs_line)

# Refinement strategies
Splines can be refined using k and p refinement. In order to do so there are two different functions
`elevate_degree(<para_dim>)` and `insert_knots(<para_dim>, <list-of-knots>)`. 

In [7]:
# Uniform refinement for a nurbs line
unique_knots = nurbs_line.unique_knots
new_knots = [.5 * (unique_knots[0][i] + unique_knots[0][i-1]) for i in range(1, len(unique_knots[0]))]
nurbs_line.insert_knots(0, new_knots)
nurbs_line.show()

In [8]:
# Order elevation
nurbs_line.elevate_degree(0)
nurbs_line.show()

### Construction
Starting from a spline, it can be used as a basis for construction using either revolutions or extrusions.

#### Extrusion

In [9]:
# Extrude along the vector [1.,0]
extruded_surface = bezier_line.create.extruded(extrusion_vector=[1.,0])
# Show the result
gus.show.show_vedo(extruded_surface, bezier_line)

#### Revolution

In [10]:
# Revolve along arbitrary axis 
revolved_surface = extruded_surface.create.revolved(
    axis=[1,0,0],
    center=[0,-1,0],
    angle=85,
    degree=True
)
# Show the result
gus.show.show_vedo(revolved_surface)

Creating Approximation.


(This creates a warning because the spline is non-rational)

In [11]:
# Revolve along arbitrary axis 
revolved_surface = extruded_surface.nurbs.create.revolved(
    axis=[1,0,0],
    center=[0,-1,0],
    angle=360,
    degree=True
)
# Show the result
gus.show.show_vedo(revolved_surface)

# Or refined
revolved_surface = extruded_surface.nurbs.create.revolved(
    axis=[1,0,0],
    center=[0,-1,0],
    angle=360,
    n_knot_spans=9,
    degree=True
)
# Show the result
gus.show.show_vedo(revolved_surface)


## Bezier Manipulations
For Microstructure construction, this is the most important aspect of gustaf.
### Multiplication
Multiplication between two splines is possible, as long as the dimensions of the physical space are compatible. Multiplication of two splines $A(t)$ and $B(t)$ results in a new spline $C$

$ A(t) * B(t) = C(t) \quad \forall t$

In [12]:
# Scalar Spline as factor
scalar_spline = gus.Bezier(
    degrees=[2],
    control_points=[[1.],[2],[1]]
)
# Vector Spline as factor
vector_spline = gus.Bezier(
    degrees=[1],
    control_points=[[0.,1.],[3.,0]]
)
# Product
product = vector_spline * scalar_spline

# Check results by evaluating at a random point
eval_query = np.random.rand(10)
assert np.allclose(
    product.evaluate([[eval_query]]), 
    (vector_spline.evaluate([[eval_query]])
     * scalar_spline.evaluate([[eval_query]]))
    )

# Print out data
_ = [[print(a + ":"),print(b)] for (a,b) in product.todict().items()]

degrees:
[3]
control_points:
[[0.         1.        ]
 [1.         1.33333333]
 [4.         0.33333333]
 [3.         0.        ]]


## Addition
As for the Multiplication, Addition of two splines $A(t)$ and $B(t)$ results in a new spline $C$

$ A(t) + B(t) = C(t) \quad \forall t$

As long as the addition of the spline types is defined.

In [13]:
# Vector Spline as factor
vector_spline = gus.Bezier(
    degrees=[1],
    control_points=[[0.,1.],[3.,0]]
)

# Create a second spline with different orders
second_spline = vector_spline.copy()
second_spline.elevate_degree(0)

# Sum
sum_spline = vector_spline + second_spline

# Print out data
_ = [[print(a + ":"),print(b)] for (a,b) in sum_spline.todict().items()]

# Check results by evuating at a random point
eval_query = np.random.rand()
assert np.allclose(
    sum_spline.evaluate([[eval_query]]), 
    (vector_spline.evaluate([[eval_query]])
     + vector_spline.evaluate([[eval_query]]))
    )

degrees:
[2]
control_points:
[[0. 2.]
 [3. 1.]
 [6. 0.]]


## Composition
Composition is in the center of microstructure construction. The result of functional composition is a new spline fulfilling 

$ A \circ B = A(B(t))= C(t) \quad \forall t$

Here, the parametric dimension of the outer (or deformation function) must match the physical dimension of the inner function.

In [14]:
# Inner function (quarter circle)
quarter_circle = gus.RationalBezier(
    degrees=[2],
    control_points=[[1,0],[1,1],[0,1]],
    weights=[1,2**-.5,1]
)

# Outer Function (Rotated Rectangle)
rectangle_surface = gus.Bezier(
    degrees=[1,1],
    control_points=[[.5,0],[1.,.5],[0.,.5],[.5,1.]]
)

# Product
composition = rectangle_surface.compose(quarter_circle)

# Show composition
gus.show.show_vedo(
    quarter_circle,
    rectangle_surface,
    [rectangle_surface,composition]
)

Or into a 3D surface

In [15]:
# Update Outer Function (Rotated Rectangle)
rectangle_surface = gus.Bezier(
    degrees=[1,1],
    control_points=[[.5,0,0],[1.,.5,.2],[0.,.5,.2],[.5,1.,0]]
)

# Product
composition = rectangle_surface.compose(quarter_circle)

# Show composition
gus.show.show_vedo(
    quarter_circle,
    rectangle_surface,
    [rectangle_surface,composition]
)

In [16]:
# Make sure the composition is correct
queries = np.random.rand(10,1)
print(
    np.allclose(
        composition.evaluate(queries),
        rectangle_surface.evaluate(quarter_circle.evaluate(queries))
    )
)

True


In [17]:
# Comparison to standard ffd
line = gus.Bezier(
        degrees=[1],
        control_points=[
            [0.2, 0.2],
            [0.8,0.4]
        ])

outer_function = gus.Bezier(
        degrees=[2,1],
        control_points=[
            [0., 0.],
            [2., 2.],
            [4., 0.],
            [0., 2.],
            [2., 4.],
            [4., 2.]
        ])

ffd_line = line.copy()
ffd_line.control_points = outer_function.evaluate(ffd_line.control_points)

composed_line = outer_function.compose(line)

# Plot results
gus.show.show_vedo(
    ['Original Line', line],
    ['Outer Function', outer_function],
    ['FFD Line', [ffd_line, outer_function]],
    ['Composed Line', [composed_line, outer_function]]
)

In [18]:
# Construct complete Microstructures
generator = gus.spline.microstructure.Generator()
generator.deformation_function = gus.Bezier(
        degrees=[2,1],
        control_points=[
            [0., 0.],
            [2., 2.],
            [4., 0.],
            [0., 2.],
            [2., 4.],
            [4., 2.]
        ])
micro_tile = [
        gus.Bezier(
                degrees=[3],
                control_points=[[0, .5], [.5, 1], [.5, 0], [1, .5]]
        ),
        gus.Bezier(
                degrees=[4],
                control_points=[
                        [0.5, 0], [0.75, .5], [0.8, .8], [0.25, 0.5], [.5, 1]
                ]
        )
]
gus.show.show_vedo(
    ['Microtile',micro_tile],
    knots=False,
    control_points=False
)

generator.microtile = micro_tile
generator.tiling = [8, 8]
generator.show(
        knots=False, control_points=False, title="2D Lattice Microstructure"
)

In [19]:
# Create a Multipatch system and export it as MFEM
rectangle = gus.Bezier(
    degrees=[1,1],
    control_points=[
        [0,0],
        [1,0],
        [0,1],
        [1,1]
    ]
).bspline

# refine
for i_pd in range(2):
    rectangle.elevate_degree(i_pd)
    rectangle.insert_knots(i_pd, [.25,.5,.75])
rectangle.show()

multi_patch_mesh = rectangle.extract.beziers()

# Boundary identifiers
def boundary0(x):
    return x[:,1] > 0.99
def boundary1(x):
    return x[:,1] < 0.01

# Export in MFEM format
gus.spline.io.mfem.export_cartesian("test_mesh.mesh",
                            multi_patch_mesh, 
                            boundary_functions=[
                                boundary0,
                                boundary1
                            ])

gus.spline.io.json.export(
        fname="test_mesh.json", 
        spline_list=multi_patch_mesh,
        base64encoding=True
        )

# Meshing and exporting
mesh_resolutions = [8, 8]
mesh_list = [m.extract.faces(resolutions=mesh_resolutions) 
        for m in multi_patch_mesh]
mesh = gus.Faces.concat(mesh_list)
mesh.merge_vertices()
# Export here
mesh.show()
